In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# ----------------------
# 1. Load model + tokenizer
# ----------------------
MODEL_NAME = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

# ----------------------
# 2. Define candidate labels
# ----------------------
candidate_labels = ["gpt-3.5-turbo", "vicuna-13b-v1.2", "alpaca-13b", "claude-v1"]

# Tokenize each label (some may split into multiple subtokens)
label_token_ids = []
for lbl in candidate_labels:
    tokens = tokenizer.tokenize(lbl.lower())  # normalize case
    ids = tokenizer.convert_tokens_to_ids(tokens)
    label_token_ids.append(ids)
    print(f"Label '{lbl}' -> tokens {tokens} -> ids {ids}")

# ----------------------
# 3. Build a few-shot prompt with [MASK]
# ----------------------
prompt = """
You are given a dataset of user prompts, model responses, and follow-up tasks.
Decide which model is the winner based on quality of response.

Query: Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.
Winner: GPT-3.5-turbo
Query: Draft a professional email seeking your supervisor's feedback on the 'Quarterly Financial Report' you prepared. Ask specifically about the data analysis, presentation style, and the clarity of conclusions drawn. Keep the email short and to the point.
Winner: GPT-3.5-turbo
Query: Write a persuasive email to convince your introverted friend, who dislikes public speaking, to volunteer as a guest speaker at a local event. Use compelling arguments and address potential objections. Please be concise.
Winner: [MASK]

"""

inputs = tokenizer(prompt, return_tensors="pt")
mask_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

# ----------------------
# 4. Forward pass
# ----------------------
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

mask_logits = logits[0, mask_index, :]  # shape: [1, vocab_size]

# ----------------------
# 5. Compute restricted probabilities
# ----------------------
def score_label(token_ids):
    """Compute mean log-prob across subtokens for multi-token labels."""
    log_probs = []
    for tid in token_ids:
        prob = torch.log_softmax(mask_logits, dim=-1)[0, tid]
        log_probs.append(prob)
    return torch.stack(log_probs).mean().item()

scores = [score_label(ids) for ids in label_token_ids]

# Normalize into probabilities
probs = torch.softmax(torch.tensor(scores), dim=-1)

# ----------------------
# 6. Print results
# ----------------------
print("\n--- Predictions ---")
for lbl, p in zip(candidate_labels, probs.tolist()):
    print(f"{lbl}: {p:.4f}")

predicted = candidate_labels[torch.argmax(probs)]
print(f"\nPredicted label: {predicted}")
